In [23]:
import cv2
import numpy as np
import csv
import os

In [3]:
def bitstouint(bits):
    uint = 0
    for bit in bits:
        uint = (uint <<1) | bit
    return uint

In [6]:
def encodeHamming(msg,n,k) :
    m = n - k
    encoded = []
    r = 0 #rth redundant bit
    
    #set up codeword loop
    for i in range(1, n+1):
        if i == 2 ** r :
            encoded.append(0)
            r += 1
        else :
            encoded.append(int(msg[i-r-1]))

    #set paritybit values
    rval = 0
    for i in range(m) :
        rval = 2 ** i
        count1 = 0
        for j in range(1,n+1) :
            if ((j & (1 << i)) & rval == rval and encoded[j-1] ==1):
                count1 += 1
            if (count1 % 2 == 1) :
                encoded[rval-1] = 1
    return encoded

In [8]:
def Hamming(encoded):
    error = 0
    idx = 1
    for i in encoded :
        if (i==1) :
            error = error ^ idx
        idx += 1
    if (error > 0 and error < idx) :
        encoded[error-1] = int(not encoded[error-1])
    r = 0
    bits = []
    for i in range(1,len(encoded)+1):
        if (i == 2 ** r) :
            r += 1
        else :
            bits.append(encoded[i-1])
    return bits

In [9]:
def Generators (n, k):
    m = n-k
    G = [[0 for j in range(n)] for i in range(k)]
    H = [[0 for j in range(n)] for i in range(m)]
    for i in range (k):
        G[i][i] = 1
    for i in range (m):
        H[i][k+i] = 1
    r = 0
    for i in range (1,n+1):
        if i == 2**r :
            r += 1
        else :
            bin = format(i,'0'+str(m)+ 'b')
            for j in range(1,m+1):
                G[i-r-1][k+j-1] = int(bin[-j])
                H[j-1][i-r-1] = int(bin[-j])
    return G, H

In [10]:
def encodeHammingM(msg,G):
    k = len(G)
    n = len(G[0])
    codeword = [0 for i in range(n)]

    #Matrix Xor Multiplication
    for i in range(n):
        for j in range(k):
            codeword[i] = codeword[i] ^ (int(msg[j]) & G[j][i])
    return codeword

In [11]:
def decodeHammingM(codeword, H):
    n = len(codeword)
    m = len(H)
    syndrome = [0 for i in range(m)]

    #Matrix Xor multiplication
    for i in range(m):
        for j in range(n):
            syndrome[i] = syndrome[i] ^ (H[i][j] & int(codeword[j]) )

    #Check any syndrome columns (error correction)
    for column in range(n):
        allsame = True
        for row in range(m):
            if (H[row][column] != syndrome[row]) :
                allsame = False
        if allsame :
            codeword[column] = int(not codeword[column])
            break
    return codeword

In [14]:
def hxx(x, y):
    size = x.shape[-1]
    px = np.histogram(x, 256, (0, 255))[0] / size
    py = np.histogram(y, 256, (0, 255))[0] / size
    hx = - np.sum(px * np.log(px + 1e-8))
    hy = - np.sum(py * np.log(py + 1e-8))
 
    hxy = np.histogram2d(x, y, 256, [[0, 255], [0, 255]])[0]
    hxy /= (1.0 * size)
    hxy = - np.sum(hxy * np.log(hxy + 1e-8))
 
    r = hx + hy - hxy
    return r

In [15]:
# 計算 psnr
def psnr(originImage,hidedImage):
   diffrence = originImage-hidedImage
   mse = np.mean(np.square(diffrence))
   psnr = 10 * np.log10(255 * 255 / mse)
   return(psnr)

In [17]:
def Hamming74Single(imageSrc,saveSrc,prt):
    image = cv2.imread(imageSrc,0)
    height = image.shape[0]
    width = image.shape[1]
    if(prt):
        print("height: "+str(height))
        print("width: "+str(width))

    codewords = [[0 for j in range(width)] for i in range(height)]
    received = [[0 for j in range(width)] for i in range(height)]

    for i in range(height):
        for j in range(width):
            codewords[i][j] =encodeHamming(format(image[i][j],'08b'), 12,8)

    for i in range(height):
        for j in range(width):
            received[i][j] = bitstouint(Hamming(codewords[i][j]))
    received = np.array(received,dtype=np.uint8)
    cv2.imwrite(saveSrc, received)

    HammingImage = cv2.imread(saveSrc)
    HammingImage = cv2.cvtColor(HammingImage,cv2.COLOR_BGR2GRAY) 
    grayscaleImage = cv2.imread(imageSrc)
    grayscaleImage = cv2.cvtColor(grayscaleImage,cv2.COLOR_BGR2GRAY) 

    if(prt):
        print('PSNR: '+str(psnr(grayscaleImage,HammingImage)))

    x = np.reshape(grayscaleImage, -1)
    y = np.reshape(HammingImage, -1)
    
    if(prt):
        print("相對熵= "+str(hxx(x, y)))

    return psnr(grayscaleImage,HammingImage),hxx(x, y)

In [18]:
Hamming74Single("../images/lennaGrayScaleCrop.jpg","../images/LennaAfterHamming74.jpg",True)

height: 480
width: 640
PSNR: 37.726545783838155
相對熵= 2.1722847693969767


(37.726545783838155, 2.1722847693969767)

In [19]:
Hamming74Single("../images/lennaGrayScaleCrop.jpg","../images/LennaAfterHamming74.jpg",False)

(37.726545783838155, 2.1722847693969767)

In [27]:
def writeCSV(array,fileName):
    with open(fileName, 'w', newline='') as csvfile:
        # 以空白分隔欄位，建立 CSV 檔寫入器
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(['Hamming74'])
        for number in array:
            writer.writerow([number])
    csvfile.close()

In [26]:
imageNamePath = '../database/resizeImageData.txt'
imageNameArray=[]
imageLength=0
file = open(imageNamePath, 'r')
for name in file.readlines():
    imageNameArray.append('../database'+name[1:-1])
file.close()
imageLength=len(imageNameArray)
print("照片數量: "+str(imageLength))

照片數量: 100


In [29]:
def Hamming74Multiple(imageNameArray,prt):
    psnrArray=[]
    hxxArray=[]
    #取得圖片陣列
    for index,imageName in enumerate(imageNameArray):
        psnr,hxx=Hamming74Single(imageName,"../database/"+str(index+1)+'Hamming74.jpg',prt)
        psnrArray.append(psnr)
        hxxArray.append(hxx)
    writeCSV(psnrArray,'./PSNR.csv')
    writeCSV(hxxArray,'./hxx.csv')

In [30]:
Hamming74Multiple(imageNameArray,False)